## 0. 데이터 준비

In [1]:
from rdkit import Chem

import pandas as pd
import numpy as np

import torch

import matplotlib.pyplot as plt

RDKit WARNING: [00:22:04] Enabling RDKit 2019.09.3 jupyter extensions


### 0.1 데이터 불러오기

In [8]:
def load_data(path):
    df_train = pd.read_csv(path + '/data/org/train_.csv')
    df_test = pd.read_csv(path + '/data/org/valid_.csv')
    
    df_train = df_train.rename(columns={'Unnamed: 0' : "idx"})
    df_test = df_test.rename(columns={'Unnamed: 0' : "idx"})
    
    df_all = df_train.append(df_test).reset_index(drop=True)
    
    return df_all, df_train, df_test

In [9]:
CURRENT_PATH = '/Users/skcc10170/Desktop'
df_all, df_train, df_test = load_data(path=CURRENT_PATH)

### 0.2 컬럼 분류하기
먼저 다음과 같이 분류할 수 있습니다.
- 스마일코드 (1개 컬럼)
    - 화합물의 구조를 문자열로 표기
- 분자의 지문 데이터 (1024개씩 3개, 3072개 컬럼)
    - ecfp : 1024개 column
    - fcfp : 1024개 column
    - ptfp : 1024개 column
- 분자자체 특성 (4개 컬럼)
    - MolWt : 화합물의 분자 질량
    - clogp : 분배 계수
    - sa_score : 합성 가능성
    - qed : 약물 유사성

In [10]:
def classify_cols(df):
    cols = df.columns

    # smiles code
    col_smiles = ['SMILES']

    # node-edge level (3 footprints)
    col_ecfp = list(cols[cols.str.contains('ecfp_')]) # ecfp 1024개
    col_fcfp = list(cols[cols.str.contains('fcfp_')]) # fcfp 1024개
    col_ptfp = list(cols[cols.str.contains('ptfp_')]) # ptfp 1024개

    # graph level
    col_mol = list(cols[-5:-1])

    # input cols
    col_input = col_ecfp + col_fcfp + col_ptfp + col_mol # col_smiles 제외

    # label
    col_label = ['label']
    
    return col_smiles[0], col_ecfp, col_fcfp, col_ptfp, col_mol, col_label[0]

In [11]:
cols = classify_cols(df_train)

### 0.3 mol2graph
분자를 그래프로 해석한다면
- 그래프(분자)
- 노드(원자) -> 노드 feature matrix
- 엣지(연결관계) -> 엣지 feature matrix (일단 생략)

3457이 제일 쉬움

In [12]:
MAX_LEN = df_all['SMILES'].apply(lambda x: Chem.MolFromSmiles(x).GetNumAtoms()).max()
LIST_SYMBOLS = list(set.union(*df_all['SMILES'].apply(
    lambda x: set([atom.GetSymbol() for atom in Chem.MolFromSmiles(x).GetAtoms()])).values))
NUM_ATOM_FEATURES = 5

In [13]:
MAX_LEN, LIST_SYMBOLS, NUM_ATOM_FEATURES

(88, ['H', 'Br', 'Na', 'Cl', 'Se', 'C', 'P', 'I', 'S', 'F', 'Si', 'O', 'N'], 5)

In [14]:
def atom_feature(atom):
    return np.array(char_to_ix(atom.GetSymbol(), LIST_SYMBOLS) +
                    char_to_ix(atom.GetDegree(), [0, 1, 2, 3, 4, 5]) +
                    char_to_ix(atom.GetTotalNumHs(), [0, 1, 2, 3, 4]) +
                    char_to_ix(atom.GetImplicitValence(), [0, 1, 2, 3, 4, 5]) +
                    char_to_ix(int(atom.GetIsAromatic()), [0, 1]))    # (40, 6, 5, 6, 2)


def char_to_ix(x, allowable_set):
    if x not in allowable_set:
        return [0] # Unknown Atom Token
    return [allowable_set.index(x)+1]

In [15]:
def mol2graph(smi):
    mol = Chem.MolFromSmiles(smi)
    num_atom = mol.GetNumAtoms()
    
    X = np.zeros((num_atom, NUM_ATOM_FEATURES), dtype=np.uint8)
    A = np.zeros((num_atom, num_atom), dtype=np.uint8)

    A = Chem.rdmolops.GetAdjacencyMatrix(
        mol).astype(np.uint8, copy=False)
    A += np.eye(num_atom, dtype=np.uint8)
    
    for idx, atom in enumerate(mol.GetAtoms()):
        feature = atom_feature(atom)
        X[idx, :] = feature
        
    bond_a, bond_b = [], []
    for bond in mol.GetBonds():
        bond_a.append(bond.GetBeginAtomIdx())
        bond_b.append(bond.GetBeginAtomIdx())
        bond_a.append(bond.GetEndAtomIdx())
        bond_b.append(bond.GetEndAtomIdx())
    edge_index = [bond_a, bond_b]
    
    return X, A, edge_index

---

## normalize data

In [16]:
# 별 차이 없어서 일단 편하게 그냥
cols_mol = cols[4]
df_train[cols[4]] = (df_train[cols_mol] / df_all[cols_mol].mean()) / df_all[cols_mol].std()
df_test[cols[4]] = (df_test[cols_mol] / df_all[cols_mol].mean()) / df_all[cols_mol].std()

In [17]:
cols_input = cols[1]+cols[2]+cols[3]
cols_label = cols[-1]
X_train, y_train = torch.as_tensor(df_train[cols_input].to_numpy()), torch.as_tensor(df_train[cols_label].to_numpy())
X_test, y_test = torch.as_tensor(df_test[cols_input].to_numpy()), torch.as_tensor(df_test[cols_label].to_numpy())

In [18]:
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader, Dataset

In [19]:
# Set the random seed for a specific number(my_seed)
my_seed = 2020
torch.manual_seed(my_seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(my_seed)
    # torch.cuda.manual_seed_all(my_seed) # for multi-gpu

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [20]:
class Toxicdataset(Dataset):
    def __init__(self, X, y):
        self.X = X.float()
        self.y = y
        self.len = X.shape[0]
    
    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [21]:
train_dataset = Toxicdataset(X_train, y_train)
test_dataset = Toxicdataset(X_test, y_test)

In [22]:
batch_size = 512
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [23]:
input_size = len(cols_input) # 1024 * 3 + 5 = 3077
hidden_size1 = 120
hidden_size2 = 80
hidden_size3 = 10

num_classes = 2
dropout_probability = 0.25

class MLP(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, hidden_size3, num_classes):
        super(MLP, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Linear(input_size, hidden_size1),
            nn.BatchNorm1d(hidden_size1),
            nn.ReLU(),
            nn.Dropout(p=dropout_probability)
            )

        self.layer2 = nn.Sequential(
            nn.Linear(hidden_size1, hidden_size2),
            nn.BatchNorm1d(hidden_size2),
            nn.ReLU(),
            nn.Dropout(p=dropout_probability)
            )

        self.layer3 = nn.Sequential(
            nn.Linear(hidden_size2, hidden_size3),
            nn.BatchNorm1d(hidden_size3),
            nn.ReLU(),
            nn.Dropout(p=dropout_probability)
            )
        
        # 마지막에 relu 추가하지 않는 이유는 cross-entropy에서 softmax를 사용하기 때문
        self.final_layer = nn.Linear(hidden_size3, num_classes)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)

        out = self.final_layer(out)
        # out = self.sigmoid(out)

        return out

In [24]:
learning_rate = 0.005

model = MLP(input_size, hidden_size1, hidden_size2, hidden_size3, num_classes)
model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=1e-3)

In [25]:
model

MLP(
  (layer1): Sequential(
    (0): Linear(in_features=3072, out_features=120, bias=True)
    (1): BatchNorm1d(120, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.25, inplace=False)
  )
  (layer2): Sequential(
    (0): Linear(in_features=120, out_features=80, bias=True)
    (1): BatchNorm1d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.25, inplace=False)
  )
  (layer3): Sequential(
    (0): Linear(in_features=80, out_features=10, bias=True)
    (1): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.25, inplace=False)
  )
  (final_layer): Linear(in_features=10, out_features=2, bias=True)
)

In [26]:
sum([p.numel() for p in model.parameters()])

379692

In [27]:
def Accuracy(output, target):
    return torch.sum(torch.max(output, dim=1)[1] == target).float() / float(target.shape[0])

def evaluate(model, test_loader):
    output_list, y_list = [], []
    total_loss = 0.
    
    for X, y in test_loader:
        X = X.to(device)
        y = y.to(device)

        output = model(X)
        output_list.extend(output)
        y_list.extend(y)
        total_loss += criterion(output, y).data.cpu().numpy()
        
    output_list = torch.stack(output_list)
    y_list = torch.stack(y_list)
        
    return total_loss / len(test_loader.dataset), Accuracy(output_list, y_list), output_list.data, y_list.data

In [28]:
num_epochs = 10

In [30]:
train_loss_arr = []
# val_loss_arr = []
test_loss_arr = []

best_ACC, final_ACC = -999., -999.
best_pred, best_y = None, None

early_stop, early_stop_max = 0., 5.

for epoch in range(num_epochs):
    
    epoch_loss = 0.
    for batch_X, batch_y in train_loader:
        # Convert numpy arrays to torch tensors
        batch_X = batch_X.to(device)
        batch_y = batch_y.to(device)
    
        # Forward Pass
        model.train()
        outputs = model(batch_X)
        train_loss = criterion(outputs, batch_y)
        epoch_loss += train_loss.data
    
        # Backward and optimize
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        
    train_loss_arr.append(epoch_loss / len(train_loader.dataset))
 
    if epoch % 10 == 0:
        model.eval()
        
        train_loss, ACC_train, _, _ = evaluate(model, train_loader)
        # val_loss, ACC_val, val_pred, val_y = evaluate(model, val_loader)
        test_loss, ACC_test, test_pred, test_y = evaluate(model, test_loader)
        
        # val_loss_arr.append(val_loss)
        test_loss_arr.append(test_loss)
        
        print('Epoch [{}/{}], Train Loss: {:.4f}, Test Loss: {:.4f}, Train ACC: {:.4f}, Test ACC: {:.4f} *'.format(epoch, num_epochs, train_loss, test_loss, ACC_train, ACC_test))

        
    #     if best_ACC < ACC_val:
    #         best_ACC = ACC_val
    #         best_pred = test_pred
    #         best_y = test_y
    #         early_stop = 0
            
    #         final_ACC = ACC_test
    #         print('Epoch [{}/{}], Train Loss: {:.4f}, Valid Loss: {:.4f}, Train ACC: {:.4f}, Valid ACC: {:.4f} *'.format(epoch, num_epochs, train_loss, val_loss, ACC_train, ACC_val))
    #     else:
    #         early_stop += 1
    #         print('Epoch [{}/{}], Train Loss: {:.4f}, Valid Loss: {:.4f}, Train ACC: {:.4f}, Valid ACC: {:.4f}'.format(epoch, num_epochs, train_loss, val_loss, ACC_train, ACC_val))   

    # if early_stop >= early_stop_max:
    #     break

Epoch [0/10], Train Loss: 0.0008, Test Loss: 0.0011, Train ACC: 0.8439, Test ACC: 0.7645 *
